In [1]:
import pandas as pd
import numpy as np
import nltk
import gensim 
from gensim.models.phrases import Phrases, Phraser
import re
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import Counter 
from gensim.models import Word2Vec
from googletrans import Translator
from gensim.models import FastText

ValueError: source code string cannot contain null bytes

In [2]:
data=pd.read_csv("Indian_songs.csv")

In [3]:
data.head()

,movieName,songName,songSinger,songMusic,songLyricist,songLyrics
0,Bala,Don’t Be Shy,"Badshah, Shalmali Kholgade, Gurdeep Mehendi",Sachin-Jigar,"Mellow D, Badshah","Sun, main hoon thoda sanki\nKarun mann ki\nBab..."
1,Laal Kaptaan,Lahu Ka Rang Kara,Samira Koppikar,Samira Koppikar,Sahib,Morey.. lahu ka rang kara\nMorey lahu ka rang ...
2,Laal Kaptaan,Red Red Najariya,Shreya Ghoshal,Samira Koppikar,Saurabh Jain,"Badnaam shehar, badnaam gully\nIsme har raat h..."
3,Laal Kaptaan,Kaal Kaal,"Brijesh Shandilya, Dino James",Samira Koppikar,Saurabh Jain,"Kaal kaal, kaal kaal, jo sapaat chal raha\nWo ..."
4,Laal Kaptaan,Taandav,"Kailash Kher, Brijesh Shandilya",Samira Koppikar,Puneet Sharma,Shor hai andher mein\nJo dher murda pedon ka\n...


In [4]:
doc=[]
for i in range(data.shape[0]):
    doc.append(nltk.word_tokenize(re.sub('[^a-zA-z\s]','',data['songLyrics'][i].lower())))

In [5]:
train=[]
strr=''
for i in doc:
    srr=""
    for j in i:
        srr=srr+' '+j
        strr=strr+' '+j
    train.append(srr)

# Preprocessing

In [6]:
def preprocess_lyrics():
    count_vec1=CountVectorizer()
    features1=count_vec1.fit_transform(train)
  
    #stemming
    stem3=['ing','ana','yan','ane','eya','ies','aaa','aan','ake']
    stem2=['on','ai','na','an','un','ey','en','se','aa']
    stem1=['a','e','i','o','u','y','n','s']
    dictt={}
    for i in count_vec1.get_feature_names():
        dictt[i]=i
    
    allwords=[]
    for i in dictt:
        allwords.append(i)


    for i in range(len(allwords)):
        j=i+1
        while j<len(allwords):
            if len(allwords[i])>3 and (len(allwords[j])-len(allwords[i])==3) and allwords[i]==allwords[j][0:-3] and (allwords[j][-3:] in stem3) and dictt[allwords[j]]==allwords[j]:
                dictt[allwords[j]]=dictt[allwords[i]]
#                 print(allwords[j],dictt[allwords[i]])

            elif len(allwords[i])>3 and (len(allwords[j])-len(allwords[i])==2) and allwords[i]==allwords[j][0:-2] and (allwords[j][-2:] in stem2) and dictt[allwords[j]]==allwords[j]:
                dictt[allwords[j]]=dictt[allwords[i]]
#                 print(allwords[j],dictt[allwords[i]])


            elif len(allwords[i])>3 and (len(allwords[j])-len(allwords[i])==1) and allwords[i]==allwords[j][0:-1] and (allwords[j][-1] in stem1) and dictt[allwords[j]]==allwords[j]:
                dictt[allwords[j]]=dictt[allwords[i]]
#                 print(allwords[j],dictt[allwords[i]])
            j=j+1

    afterstemming=[]
    for i in dictt:
        afterstemming.append(dictt[i])
    afterstemming=set(afterstemming)
    print('length after stemming', len(afterstemming))
    
    stemdictt={}
    for i in dictt:
        stemdictt[dictt[i]]=dictt[i]
    
    #spelling errors
    for i in afterstemming:
        if 'aa' in i:
            word=i.replace('aa','a')
            if word in afterstemming:
                stemdictt[i]=word

        if 'ee' in i:
            word=i.replace('ee','i')
            if word in afterstemming:
                stemdictt[i]=word

        if 'w' in i:
            word=i.replace('w','v')
            if word in afterstemming:
                stemdictt[i]=word

        if 'j' in i:
            word=i.replace('j','z')
            if word in afterstemming:
                stemdictt[i]=word

        if 'oo' in i:
            word=i.replace('oo','u')
            if word in afterstemming:
                stemdictt[i]=word
#         if stemdictt[i]!=i:
#             print(i, stemdictt[i])
            
    afterspell=[]
    
    for i in stemdictt:
        afterspell.append(stemdictt[i])
    afterspell=set(afterspell)
    print('length after spell', len(afterspell))
    
    
    for i in dictt:
        dictt[i]=stemdictt[dictt[i]]
    return dictt

In [7]:
vocab_d=preprocess_lyrics()

length after stemming 24376
length after spell 22374


In [8]:
print(len(vocab_d))
vocab_d['chhori']='chhori'
vocab_d['munda']='munda'
vocab_d['mundey']='mundey'
vocab_d['chhora']='chhora'

31402


docp is preprocessed documents list

In [9]:
docp=[]
for i in range(len(doc)):
    k=[]
    for j in range(len(doc[i])):
        if doc[i][j] in vocab_d:
            k.append(vocab_d[doc[i][j]])
        else:
            k.append(doc[i][j])
#         print(vocab_d[doc[i][j]])
    docp.append(k)

In [10]:
female_names=['ladki','girl','gori','lady','kudi','chhori','woman']
male_names=['ladka','munda','mundey','boy','chokra','chhora','man']

color =['sanwali','saanwala','pink','pinky','red','laal','kaala','kaali','gori','gora','white','black','yellow','brown']
softAttitude=['bholi','bhola','komalkomal','heeran','hirni','nadaan','beautiful','mastani','mastana','seedhi','seedha','sharmili','sharmeela','sohni','sohna','bhali','bhala']
strongAttitude=['kukkad','bigda','bigdi','khatra','khauf','handa','jungli','badmash','gussa']
cars=['car', 'gaddi','drive','lamborghini','jaguar','gaadi','motorcycle']
clothes=['jeans','skirt','shirt','lehnga','chunni','ainak','ghagra','kurta','pajama','jacket','choodi','jhumka','chasma','chashma','kangan','top']
food=['namkeen','mithi','tikhi','teekha','khatti','makkhan','sweet','nimbu','imli','mitthe','rasmalai','mirchi','mishti','naariyal']
alcohol=['daaru','whisky','daru','pila','botal','peg','shots','drink','peeta']
bodylooks=['choti','chota','cheeks','adayein','thumka','aankhen','aankhein','nazron','charming']

In [11]:
att1=[color,softAttitude,strongAttitude,cars, clothes,food,alcohol,bodylooks]

attp=[]
for i in range(len(att1)):
    k=[]
    for j in range(len(att1[i])):
        if att1[i][j] in vocab_d:
            k.append(vocab_d[att1[i][j]])
        else:
            k.append(att1[i][j])
    attp.append(list(set(k)))
color =attp[0]
softAttitude=attp[1]
strongAttitude=attp[2]
cars=attp[3]
clothes=attp[4]
food=attp[5]
alcohol=attp[6]
bodylooks=attp[7]
att1=[color,softAttitude,strongAttitude,cars, clothes,food,alcohol,bodylooks]

# weat Test

In [15]:
def weat_test(target_one,target_two, target_one_words, attribute_one,attribute_two, attribute_one_words, target_two_words, attribute_two_words):
    cos=[]
    s=0
    s1=[]
    s2=[]
    S=[]
    n=0
    for i in range(0, len(target_one_words)):
            c1=[]
            c2=[]
            for k in range(0, len(attribute_one_words)):
                wt = target_one_words[i]
                at1 = attribute_one_words[k]
                try:
                    cos1= model.wv.similarity(wt, at1)
                    cos.append(cos1)
                    c1.append(cos1)
                except:
                    cos1=0
                    cos.append(cos1)
                    c1.append(cos1)
                    continue
            for k in range(0, len(attribute_two_words)):
                cos2=0
                wt = target_one_words[i]
                at2 = attribute_two_words[k]
                try:
                    cos2= model.wv.similarity(wt, at2)
                    cos.append(cos2)
                    c2.append(cos2)
                except:
                    cos2=0
                    cos.append(cos2)
                    c2.append(cos2)
                    continue
            s1.append((np.mean(c1)-np.mean(c2)))
            S.append((np.mean(c1)-np.mean(c2)))
            n=n+1
    for i in range(0, len(target_two_words)):
            c1=[]
            c2=[]
            for k in range(0, len(attribute_one_words)):
                wt = target_two_words[i]
                at1 = attribute_one_words[k]
                try:
                    cos1= model.wv.similarity(wt, at1)
                    cos.append(cos1)
                    c1.append(cos1)
                except:
                    cos1=0
                    cos.append(cos1)
                    c1.append(cos1)
                    continue
            for k in range(0, len(attribute_two_words)):
                cos2=0
                wt = target_two_words[i]
                at2 = attribute_two_words[k]
                try:
                    cos2= model.wv.similarity(wt, at2)
                    cos.append(cos2)
                    c2.append(cos2)
                except:
                    cos2=0
                    cos.append(cos2)
                    c2.append(cos2)
                    continue
            s2.append((np.mean(c1)-np.mean(c2)))
            S.append((np.mean(c1)-np.mean(c2)))
    s=np.sum(s1)-np.sum(s2)
    stdev=np.std(S)
    print(target_one + ' vs ' + target_two  + ' , ' +attribute_one + ' vs ' + attribute_two +', d = ' + str(s/(stdev*n)))

# Word2vec using gensim


In [16]:
model = gensim.models.Word2Vec(docp,size=100,window=20,min_count=5,workers=10,iter=10) 

In [17]:
weat_test('female_names','male_names', female_names, 'softAttitude' ,'strongAttitude', softAttitude, male_names, strongAttitude)

female_names vs male_names , softAttitude vs strongAttitude, d = 0.7009549405126815


# FastText

In [23]:
model = FastText(docp, size=100, window=20, min_count=5, workers=4,sg=1)

In [24]:
weat_test('female_names','male_names', female_names, 'softAttitude' ,'strongAttitude', softAttitude, male_names, strongAttitude)

female_names vs male_names , softAttitude vs strongAttitude, d = 0.8720205802435862


# Support and Confidence

In [25]:
# entrire lyrics as doc

In [26]:
mctr=0
for d in docp:
    flag=0
    for word in male_names:
        if word in d:
            flag=1
    
    if flag==1:
        mctr=mctr+1
mctr

426

In [27]:
fctr=0
for d in docp:
    flag=0
    for word in female_names:
        if word in d:
            flag=1
    
    if flag==1:
        fctr=fctr+1
fctr

309

In [28]:
total=len(doc)

In [29]:
ctr=0
for d in docp:
    flag=0
    for word in female_names:
        for w2 in alcohol:
            if word in d:
                if w2 in d:
                    flag=1
        
    if flag==1:
        ctr=ctr+1
        
ctr

66

In [30]:
att=['color','softAttitude','strongAttitude','cars','clothes','food','alcohol','bodylooks']
fem_support=[]
male_support=[]
fem_confidence=[]
male_confidence=[]

In [31]:
att1=[color,softAttitude,strongAttitude,cars, clothes,food,alcohol,bodylooks]

In [32]:
for i in att1:
    ctr=0
    for d in docp:
        flag=0
        for word in female_names:
            for w2 in i:
                if word in d:
                    if w2 in d:
                        flag=1
        
        if flag==1:
            ctr=ctr+1
    fem_support.append((ctr/total)*100)

In [33]:
for i in att1:
    ctr=0
    for d in docp:
        flag=0
        for word in male_names:
            for w2 in i:
                if word in d:
                    if w2 in d:
                        flag=1
        
        if flag==1:
            ctr=ctr+1
    male_support.append((ctr/total)*100)

In [34]:
for i in att1:
    ctr=0
    for d in docp:
        flag=0
        for word in female_names:
            for w2 in i:
                if word in d:
                    if w2 in d:
                        flag=1
        
        if flag==1:
            ctr=ctr+1
    fem_confidence.append((ctr/fctr)*100)

In [35]:
for i in att1:
    ctr=0
    for d in docp:
        flag=0
        for word in male_names:
            for w2 in i:
                if word in d:
                    if w2 in d:
                        flag=1
        
        if flag==1:
            ctr=ctr+1
    male_confidence.append((ctr/mctr)*100)

In [37]:
support=pd.DataFrame({'attribute':att,'female_names':fem_support,'male_names':male_support})

In [38]:
support

,attribute,female_names,male_names
0,color,3.666997,2.940205
1,softAttitude,3.270565,3.766105
2,strongAttitude,0.825900,1.024116
3,cars,1.123224,1.057152
4,clothes,1.189296,1.519656
5,food,0.660720,0.726792
6,alcohol,2.180377,2.411629
7,bodylooks,2.973241,3.633961


In [39]:
confidence=pd.DataFrame({'attribute':att,'female_names':fem_confidence,'male_names':male_confidence})

In [40]:
confidence

,attribute,female_names,male_names
0,color,35.922330,20.892019
1,softAttitude,32.038835,26.760563
2,strongAttitude,8.090615,7.276995
3,cars,11.003236,7.511737
4,clothes,11.650485,10.798122
5,food,6.472492,5.164319
6,alcohol,21.359223,17.136150
7,bodylooks,29.126214,25.821596


In [ ]:
# support confidence on sentences

In [41]:
doc_sent=[]
for i in range(data.shape[0]):
    doc_sent.append(re.sub('[^a-zA-z\s]','',data['songLyrics'][i].lower()).split("\n"))
nonrep_doc=[]
for song in doc_sent:
    d=[]
    for line in song:
        if line not in d:
            d.append(line)
    nonrep_doc.append(d)

In [42]:
doca=[]
for i in nonrep_doc:
    for j in i:
        if j =='':
            continue
        doca.append(nltk.word_tokenize(j))

In [43]:
doca[0]

['sun', 'main', 'hoon', 'thoda', 'sanki']

In [44]:
doc1=[]

for l in range(len(doca)):
    k=[]
    for i in range(len(doca[l])):
        if doca[l][i] in vocab_d:
            k.append(vocab_d[doca[l][i]])
        else:
            k.append(doca[l][i])
    doc1.append(k)

In [45]:
mctr=0
for d in doc1:
    flag=0
    for word in male_names:
        if word in d:
            flag=1
    
    if flag==1:
        mctr=mctr+1
fctr=0
for d in doc1:
    flag=0
    for word in female_names:
        if word in d:
            flag=1
    
    if flag==1:
        fctr=fctr+1
mctr,fctr

(660, 544)

In [46]:
total=len(doc1)

In [47]:
ctr=0
for d in doc1:
    flag=0
    for word in female_names:
        for w2 in alcohol:
            if word in d:
                if w2 in d:
                    flag=1
        
    if flag==1:
        ctr=ctr+1
        
ctr

3

In [48]:
att=['color','softAttitude','strongAttitude','cars','clothes','food','alcohol','bodylooks']
fem_support=[]
male_support=[]
fem_confidence=[]
male_confidence=[]

In [49]:
att1=[color,softAttitude,strongAttitude,cars, clothes,food,alcohol,bodylooks]

In [50]:
for i in att1:
    ctr=0
    for d in doc1:
        flag=0
        for word in female_names:
            for w2 in i:
                if word in d:
                    if w2 in d:
                        flag=1
        
        if flag==1:
            ctr=ctr+1
    fem_support.append((ctr/total)*100)

In [51]:
for i in att1:
    ctr=0
    for d in doc1:
        flag=0
        for word in male_names:
            for w2 in i:
                if word in d:
                    if w2 in d:
                        flag=1
        
        if flag==1:
            ctr=ctr+1
    male_support.append((ctr/total)*100)

In [52]:
for i in att1:
    ctr=0
    for d in doc1:
        flag=0
        for word in female_names:
            for w2 in i:
                if word in d:
                    if w2 in d:
                        flag=1
        
        if flag==1:
            ctr=ctr+1
    fem_confidence.append((ctr/fctr)*100)

In [53]:
for i in att1:
    ctr=0
    for d in doc1:
        flag=0
        for word in male_names:
            for w2 in i:
                if word in d:
                    if w2 in d:
                        flag=1
        
        if flag==1:
            ctr=ctr+1
    male_confidence.append((ctr/mctr)*100)

In [54]:
support1=pd.DataFrame({'attribute':att,'female_names':fem_support,'male_names':male_support})

In [55]:
support1

,attribute,female_names,male_names
0,color,0.100368,0.005425
1,softAttitude,0.032552,0.012207
2,strongAttitude,0.001356,0.009494
3,cars,0.000000,0.000000
4,clothes,0.004069,0.005425
5,food,0.002713,0.000000
6,alcohol,0.004069,0.002713
7,bodylooks,0.005425,0.002713


In [56]:
confidence1=pd.DataFrame({'attribute':att,'female_names':fem_confidence,'male_names':male_confidence})

In [57]:
confidence1

,attribute,female_names,male_names
0,color,13.602941,0.606061
1,softAttitude,4.411765,1.363636
2,strongAttitude,0.183824,1.060606
3,cars,0.000000,0.000000
4,clothes,0.551471,0.606061
5,food,0.367647,0.000000
6,alcohol,0.551471,0.303030
7,bodylooks,0.735294,0.303030


In [58]:
# support confidence paragraph wise

In [59]:
doc2=[]
for i in range(data.shape[0]):
    doc2.append(re.sub('[^a-zA-z\s]','',data['songLyrics'][i].lower()).split("\n\n"))
doc22=[]
for song in doc2:
    d=[]
    for line in song:
        if line not in d:
            d.append(line)
    doc22.append(d)

In [60]:
doc222=[]
for i in doc22:
    for j in i:
        doc222.append(j.split('\n'))

In [61]:
doca=[]
for i in range(len(doc222)):
    doca.append(nltk.word_tokenize(" ".join(doc222[i])))

In [62]:
doc1=[]

for l in range(len(doca)):
    k=[]
    for i in range(len(doca[l])):
        if doca[l][i] in vocab_d:
            k.append(vocab_d[doca[l][i]])
        else:
            k.append(doca[l][i])
    doc1.append(k)

In [63]:
mctr=0
for d in doc1:
    flag=0
    for word in male_names:
        if word in d:
            flag=1
    
    if flag==1:
        mctr=mctr+1
fctr=0
for d in doc1:
    flag=0
    for word in female_names:
        if word in d:
            flag=1
    
    if flag==1:
        fctr=fctr+1
mctr,fctr

(462, 350)

In [64]:
total=len(doc1)

In [65]:
ctr=0
for d in doc1:
    flag=0
    for word in female_names:
        for w2 in alcohol:
            if word in d:
                if w2 in d:
                    flag=1
        
    if flag==1:
        ctr=ctr+1
        
ctr

59

In [66]:
att=['color','softAttitude','strongAttitude','cars','clothes','food','alcohol','bodylooks']
fem_support=[]
male_support=[]
fem_confidence=[]
male_confidence=[]
att1=[color,softAttitude,strongAttitude,cars, clothes,food,alcohol,bodylooks]

In [67]:
for i in att1:
    ctr=0
    for d in doc1:
        flag=0
        for word in female_names:
            for w2 in i:
                if word in d:
                    if w2 in d:
                        flag=1
        
        if flag==1:
            ctr=ctr+1
    fem_support.append((ctr/total)*100)

In [68]:
for i in att1:
    ctr=0
    for d in doc1:
        flag=0
        for word in male_names:
            for w2 in i:
                if word in d:
                    if w2 in d:
                        flag=1
        
        if flag==1:
            ctr=ctr+1
    male_support.append((ctr/total)*100)

In [69]:
for i in att1:
    ctr=0
    for d in doc1:
        flag=0
        for word in female_names:
            for w2 in i:
                if word in d:
                    if w2 in d:
                        flag=1
        
        if flag==1:
            ctr=ctr+1
    fem_confidence.append((ctr/fctr)*100)

In [70]:
for i in att1:
    ctr=0
    for d in doc1:
        flag=0
        for word in male_names:
            for w2 in i:
                if word in d:
                    if w2 in d:
                        flag=1
        
        if flag==1:
            ctr=ctr+1
    male_confidence.append((ctr/mctr)*100)

In [71]:
support1=pd.DataFrame({'attribute':att,'female_names':fem_support,'male_names':male_support})

In [72]:
support1

,attribute,female_names,male_names
0,color,1.657545,1.282252
1,softAttitude,1.391712,1.563722
2,strongAttitude,0.344019,0.469116
3,cars,0.531665,0.437842
4,clothes,0.531665,0.641126
5,food,0.281470,0.250195
6,alcohol,0.922596,0.938233
7,bodylooks,1.282252,1.501173


In [73]:
confidence1=pd.DataFrame({'attribute':att,'female_names':fem_confidence,'male_names':male_confidence})

In [74]:
confidence1

,attribute,female_names,male_names
0,color,30.285714,17.748918
1,softAttitude,25.428571,21.645022
2,strongAttitude,6.285714,6.493506
3,cars,9.714286,6.060606
4,clothes,9.714286,8.874459
5,food,5.142857,3.463203
6,alcohol,16.857143,12.987013
7,bodylooks,23.428571,20.779221
